In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['descripcion_longitud_media_de_palabra', 'metrostotales',
       'metros_x_espacio', 'idzona', 'lat', 'titulo_longitud_media_de_palabra',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial', 'metros_x_garages',
       'metroscubiertos', 'metros_x_habitaciones', 'metros_x_banos',
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'titulo_cantidad_caracteres_en_palabras_importantes',
       'descripcion_cantidad_palabras_top_k',
       'descripcion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_provincia',
       'descripcion_cantidad_stopwords',
       'descripcion_cantidad_signos_puntacion',
       'direccion_longitud_media_de_palabra',
       'titulo_palabras_top_k_sobre_total_palabras',
       'habitaciones_preciopromedio_ciudad', 'antiguedad', 'lng',
       'garages_preciopromedio_ciudad',
       'direccion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_ciudad',
       'banos_preciopromedio_ciudad', 'descripcion_cantidad_prefijos_top_k',
       'precio_x_m2', 'año', 'titulo_cantidad_fijos_top_k',
       'descripcion_cantidad_palabras_importantes',
       'descripcion_cantidad_postfijos_top_k', 'ciudad_mean_mean_todas',
       'ciudad_mean_antiguedad_sobre_mean_metrocubiertos',
       'habitaciones_preciopromedio_metroscubiertos', 'cantidad_espacios',
       'direccion_cantidad_al_menos_una_mayuscula',
       'direccion_cantidad_fijos_top_k', 'ciudad_distancia_al_origen']

In [0]:
X = train[FEATURES]
y = train[TARGET]

In [0]:
K = 4
def objective(hyper_parametros):

    xgb_regressor = XGBRegressor(objective = 'reg:squarederror',**hyper_parametros)
    
    score = cross_val_score(xgb_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    "n_estimators": hp.randint("xgb_re__n_estimators", 200) + 400, 
    "learning_rate": hp.uniform("xgb_re__learning_rate", 0.01, 0.03), 
    "gamma": hp.uniform("xgb_re__gamma", 2, 4),
    "lambda": hp.uniform("xgb_re__lambda", 2, 4),
    "max_depth": hp.randint("xgb_re__max_depth", 4) + 16, 
    "min_child_weight": hp.randint("xgb_re__min_child_weight", 1000) + 2,
    'colsample_bytree': 0.8,
}

In [0]:
N_ITER = 20
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?]

SCORE: -519440.767 params {'colsample_bytree': 0.8, 'gamma': 4.7506160486034, 'lambda': 1.1183589932730134, 'learning_rate': 0.02611652524078614, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 604, 'subsample': 0.8}  

SCORE: -578336.292 params {'gamma': 3.9592400418765896, 'lambda': 3.7750522524494228, 'learning_rate': 0.015465931263479917, 'max_depth': 18, 'min_child_weight': 344, 'n_estimators': 586} 
 